-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="0c4e00b2-1879-4e05-8bad-70f2e471691c"/>


# Managing Permissions for Databases, Tables, and Views

The instructions as detailed below are provided for groups of users to explore how Table ACLs on Databricks work. It leverages Databricks SQL and the Data Explorer to accomplish these tasks, and assumes that at least one user in the group has administrator status (or that an admin has previously configured permissions to allow proper permissions for users to create databases, tables, and views). 

As written, these instructions are for the admin user to complete. The following notebook will have a similar exercise for users to complete in pairs.

## Learning Objectives
By the end of this lesson, you should be able to:
* Describe the default permissions for users and admins in DBSQL
* Identify the default owner for databases, tables, and views created in DBSQL and change ownership
* Use Data Explorer to navigate relational entities
* Configure permissions for tables and views with Data Explorer
* Configure minimal permissions to allow for table discovery and querying

In [0]:
%run ../Includes/Classroom-Setup-11.1

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| removing the working directory "dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02"

Validating the locally installed datasets:
| listing local files...(8 seconds)
| completed (8 seconds total)

Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/_checkpoints

Setup completed (9 seconds)


<i18n value="86552e2b-b893-4633-990e-4dd27255daa9"/>


## Generate Setup Statements

The following cell uses Python to extract username of the current user and format this into several statements used to create databases, tables, and views.

Only the admin needs to execute the following cell. Successful execution will print out a series of formatted SQL queries, which can be copied into the DBSQL query editor and executed.

In [0]:
DA.generate_users_table()

CREATE DATABASE IF NOT EXISTS munirsheikhcloudseekho_0lj9_da_dewd
LOCATION 'dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/database.db';

USE munirsheikhcloudseekho_0lj9_da_dewd;

CREATE TABLE users (id INT, name STRING, value DOUBLE, state STRING);

INSERT INTO users
VALUES (1, "Yve", 1.0, "CA"),
 (2, "Omar", 2.5, "NY"),
 (3, "Elia", 3.3, "OH"),
 (4, "Rebecca", 4.7, "TX"),
 (5, "Ameena", 5.3, "CA"),
 (6, "Ling", 6.6, "NY"),
 (7, "Pedro", 7.1, "KY");

CREATE VIEW ny_users_vw
AS SELECT * FROM users WHERE state = 'NY';

<i18n value="0369413a-e3d4-4fe1-8948-143fb950584b"/>


Steps:
1. Run the cell above
1. Copy the entire output to your clipboard
1. Navigate to the Databricks SQL workspace
1. Make sure that a SQL Warehouse is running
1. Use the left sidebar to select the **SQL Editor**
1. Paste the query above and click the blue **Run** in the top right

**NOTE**: You will need to be connected to a SQL warehouse to execute these queries successfully. If you cannot connect to a SQL warehouse, you will need to contact your administrator to give you access.

<i18n value="c0629565-2278-4384-9919-f3d6f946eb11"/>


## Using Data Explorer

* Use the left sidebar navigator to select the **Data** tab; this places you in the **Data Explorer**

## What is the Data Explorer?

The data explorer allows users and admins to:
* Navigate databases, tables, and views
* Explore data schema, metadata, and history
* Set and modify permissions of relational entities

Note that at the moment these instructions are being written, Unity Catalog is not yet generally available. The 3 tier namespacing functionality it adds can be previewed to an extent by switching between the default **`hive_metastore`** and the **`sample`** catalog used for example dashboards and queries. Expect the Data Explorer UI and functionality to evolve as Unity Catalog is added to workspaces.

## Configuring Permissions

By default, admins will have the ability to view all objects registered to the metastore and will be able to control permissions for other users in the workspace. Users will default to having **no** permissions on anything registered to the metastore, other than objects that they create in DBSQL; note that before users can create any databases, tables, or views, they must have create and usage privileges specifically granted to them.

Generally, permissions will be set using Groups that have been configured by an administrator, often by importing organizational structures from SCIM integration with a different identity provider. This lesson will explore Access Control Lists (ACLs) used to control permissions, but will use individuals rather than groups.

## Table ACLs

Databricks allows you to configure permissions for the following objects:

| Object | Scope |
| --- | --- |
| CATALOG | controls access to the entire data catalog. |
| DATABASE | controls access to a database. |
| TABLE | controls access to a managed or external table. |
| VIEW | controls access to SQL views. |
| FUNCTION | controls access to a named function. |
| ANY FILE | controls access to the underlying filesystem. Users granted access to ANY FILE can bypass the restrictions put on the catalog, databases, tables, and views by reading from the file system directly. |

**NOTE**: At present, the **`ANY FILE`** object cannot be set from Data Explorer.

## Granting Privileges

Databricks admins and object owners can grant privileges according to the following rules:

| Role | Can grant access privileges for |
| --- | --- |
| Databricks administrator | All objects in the catalog and the underlying filesystem. |
| Catalog owner | All objects in the catalog. |
| Database owner | All objects in the database. |
| Table owner | Only the table (similar options for views and functions). |

**NOTE**: At present, Data Explorer can only be used to modify ownership of databases, tables, and views. Catalog permissions can be set interactively with the SQL Query Editor.

## Privileges

The following privileges can be configured in Data Explorer:

| Privilege | Ability |
| --- | --- |
| ALL PRIVILEGES | gives all privileges (is translated into all the below privileges). |
| SELECT | gives read access to an object. |
| MODIFY | gives ability to add, delete, and modify data to or from an object. |
| READ_METADATA | gives ability to view an object and its metadata. |
| USAGE | does not give any abilities, but is an additional requirement to perform any action on a database object. |
| CREATE | gives ability to create an object (for example, a table in a database). |
| CREATE_NAMED_FUNCTION | gives ability to create a named UDF in an existing catalog or schema. |

<i18n value="e82e2a15-0fdd-4af1-9383-9681a2cddbb8"/>


## Review the Default Permissions
In the Data Explorer, find the database you created earlier (this should follow the pattern **`dbacademy_<username>_acls_demo`**).

Clicking on the database name should display a list of the contained tables and views on the left hand side. On the right, you'll see some details about the database, including the **Owner** and **Location**. 

Click the **Permissions** tab to review who presently has permissions (depending on your workspace configuration, some permissions may have been inherited from settings on the catalog).

<i18n value="87339040-a0f3-4889-bc16-065f1c8a25b6"/>


## Assigning Ownership

Click the blue pencil next to the **Owner** field. Note that an owner can be set as an individual OR a group. For most implementations, having one or several small groups of trusted power users as owners will limit admin access to important datasets while ensuring that a single user does not create a choke point in productivity.

Here, we'll set the owner to **Admins**, which is a default group containing all workspace administrators.

<i18n value="86f51802-f151-4fe1-8835-a793a3ce5756"/>


## Change Database Permissions

Begin by allowing all users to review metadata about the database.

Steps:
1. Make sure you have the **Permissions** tab selected for the database
1. Click the blue **Grant** button
1. Select the **USAGE** and **READ_METADATA** options
1. Select the **All Users** group from the drop down menu at the top
1. Click **OK**

Note that users may need to refresh their view to see these permissions updated. Updates should be reflected for users in near real time for both the Data Explorer and the SQL Editor.

<i18n value="f3026810-bf59-4558-bfd0-eb46636ceafb"/>


## Change View Permissions

While users can now see information about this database, they won't be able to interact with the table of view declared above.

Let's start by giving users the ability to query our view.

Steps:
1. Select the **`ny_users_vw`**
1. Select the **Permissions** tab
   * Users should have inherited the permissions granted at the database level; you'll be able to see which permissions users currently have on an asset, as well as where that permission is inherited from
1. Click the blue **Grant** button
1. Select the **SELECT** and **READ_METADATA** options
   * **READ_METADATA** is technically redundant, as users have already inherited this from the database. However, granting it at the view level allows us to ensure users still have this permission even if the database permissions are revoked
1. Select the **All Users** group from the drop down menu at the top
1. Click **OK**

<i18n value="957dfb39-747b-45f3-86c3-c6d23496e19d"/>


## Run a Query to Confirm

In the **SQL Editor**, all users should use the **Schema Browser** on the lefthand side to navigate to the database being controlled by the admin.

Users should start a query by typing **`SELECT * FROM`** and then click the **>>** that appears while hovering over the view name to insert it into their query.

This query should return 2 results.

**NOTE**: This view is defined against the **`users`** table, which has not had any permissions set yet. Note that users have access only to that portion of the data that passes through the filters defined on the view; this pattern demonstrates how a single underlying table can be used to drive controlled access to data for relevant stakeholders.

<i18n value="5fece6bf-b060-46b3-bf10-1052baf1b6fe"/>


## Change Table Permissions

Perform the same steps as above, but now for the **`users`** table.

Steps:
1. Select the **`users`** table
1. Select the **Permissions** tab
1. Click the blue **Grant** button
1. Select the **SELECT** and **READ_METADATA** options
1. Select the **All Users** group from the drop down menu at the top
1. Click **OK**

<i18n value="77c59af7-3bcc-4e46-97c2-64bd3dc77d34"/>


## Have Users Attempt to **`DROP TABLE`**

In the **SQL Editor**, encourage users to explore the data in this table.

Encourage users to try to modify the data here; assuming permissions were set correctly, these commands should error out.

<i18n value="bc4d8d86-00a5-416e-a482-c843e58052d2"/>


## Create a Database for Derivative Datasets

In most cases users will need a location to save out derivative datasets. At present, users may not have the ability to create new tables in any location (depending on existing ACLs in the workspace and databases created during previous lessons students have completed).

The cell below prints out the code to generate a new database and grant permissions to all users.

**NOTE**: Here we set permissions using the SQL Editor rather than the Data Explorer. You can review the Query History to note that all of our previous permission changes from Data Explorer were executed as SQL queries and logged here (additionally, most actions in the Data Explorer are logged with the corresponding SQL query used to populate the UI fields).

In [0]:
DA.generate_create_database_with_grants()

CREATE DATABASE munirsheikhcloudseekho_0lj9_da_dewd_derivative;

GRANT USAGE, READ_METADATA, CREATE, MODIFY, SELECT ON DATABASE `munirsheikhcloudseekho_0lj9_da_dewd_derivative` TO `users`;

SHOW GRANT ON DATABASE `munirsheikhcloudseekho_0lj9_da_dewd_derivative`;

<i18n value="5f0e0657-1cf3-4728-9a31-07c0d57199e4"/>


## Have Users Create New Tables or Views

Give users a moment to test that they can create tables and views in this new database.

**NOTE**: because users were also granted **MODIFY** and **SELECT** permissions, all users will immediately be able to query and modify entities created by their peers.

<i18n value="9e468552-03bc-4c5a-b0e7-fcf5052cc478"/>


## Admin Configuration

At present, users do not have any Table ACL permissions granted on the default catalog **`hive_metastore`** by default. The next lab assumes that users will be able to create databases.

To enable the ability to create databases and tables in the default catalog using Databricks SQL, have a workspace admin run the following command in the DBSQL query editor:

<strong><code>GRANT usage, create ON CATALOG &#x60;hive_metastore&#x60; TO &#x60;users&#x60;</code></strong>

To confirm this has run successfully, execute the following query:

<strong><code>SHOW GRANT ON CATALOG &#x60;hive_metastore&#x60;</code></strong>

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>